# Bell with clapper

In this section, we model the clapper moving in response to the bell. In this version, the bell moves independently, and is unaffected by the motion of the clapper.

In [13]:
%matplotlib notebook
from scipy.special import ellipk
import numpy as np
import matplotlib.pyplot as plt
from IPython.core.debugger import set_trace
from functools import partial
from ipywidgets import interactive, FloatSlider, Layout
#from bqplot import pyplot as plt

Some constants. 

The clapper bounds are the angles where the clapper strikes the bell. We've set these
to +/- 30 degrees here somewhat arbitarily.

In [14]:
# acceleration due to gravity
g = 9.81

# clapper bounds
phi0 = -30 * np.pi / 180
phi1 = 30 * np.pi / 180

The equations of motion are specified by the derivatives of the variables.
In this case, we include the clapper which moves in response to the bell,
but we *do not* yet include the back-action of the clapper on the bell.

In [15]:
def deriv(l_b, l_c, c, alpha, y, t):
    [theta, vtheta, phi, vphi] = y
    theta_dot  = vtheta
    vtheta_dot = - (g / l_b) * np.sin(theta)

    phi_dot = vphi

    # note sign error in the original notes 
    vphi_dot = -vtheta_dot * (1 + (c / l_c) * np.cos(phi - alpha)) \
               + (c / l_c) * (vtheta**2) * np.sin(phi - alpha) - (g / l_c) * np.sin(theta+phi)
    return np.array([theta_dot, vtheta_dot, phi_dot, vphi_dot])

Because we need to clip the clapper position at the side of the bell, we cannot use the NumPy
built-in ODE solver. So we use here a custom 4th order Runge-Kutta. Note the rk4() function is completely generic - the integrate_rk4 function takes the clapper into account

In [16]:
def rk4(f, x, t, step):
    k1 = step * f(x, t)
    x1 = x + 0.5*k1
    k2 = step * f(x1, t+0.5*step)
    x2 = x + 0.5*k2
    k3 = step * f(x2, t+0.5*step)
    x3 = x + k3;
    k4 = step * f(x3, t+step)
    return (k1 + 2*k2 + 2*k3 + k4) / 6.0

In [17]:
def integrate_rk4_clapper(y0, t, step, l_b, l_c, c, alpha, clapper):
    y = y0 + rk4(partial(deriv, l_b, l_c, c, alpha), y0, t, step)
    # check for clapper clapping the bell
    if y[2] > phi1:
        clapper = True
        y[2] = phi1
        y[3] = 0
    elif y[2] < phi0:
        clapper = True
        y[2] = phi0
        y[3] = 0
    else:
        clapper = False
    return [clapper, y]

In [18]:
def integrate_sequence(y0, tindex, theta_0, l_b, l_c, c, alpha):
    y = [np.array([theta_0, 0.0, phi1, 0.0])]
    StrikeTimes = []
    t_last = tindex[0]
    clapper = True
    for t in tindex[1:]:
        step = t - t_last
        OldClapper = clapper
        [clapper, yNext] = integrate_rk4_clapper(y[-1], t_last, step, l_b, l_c, c, alpha, clapper)
        y.append(yNext)
        if clapper and not OldClapper:
            StrikeTimes.append(t)
        t_last = t
    return [StrikeTimes, np.array(y)]

In [19]:
# period of small oscillations
def small_oscillation(l_b):
    return 2 * np.pi * np.sqrt(l_b / g)

# Exact formula from elliptic integral for the period
def exact_period(l_b, theta_0):
    return 4 * np.sqrt(l_b / g) * ellipk(np.sin(theta_0/2)**2)

In [20]:
def update(theta_0, l_b, l_c, c, alpha):
    Texact = exact_period(l_b, theta_0 * np.pi/180)
    t = np.linspace(0, 2.5*Texact + 1e-3, 1000)
    y0 = np.array([theta_0*np.pi/180, 0, phi1, 0])
    [StrikeTimes, y] = integrate_sequence(y0, t, theta_0*np.pi/180, l_b, l_c, c, alpha*np.pi/180)
    plt.figure(figsize=(8,6), dpi=80)
    plt.plot(t, y[:,0] * 180 / np.pi, 'b', label='bell')
    plt.plot(t, y[:,2] * 180 / np.pi, 'g', label='clapper')
    plt.plot([], [], 'r-', label='strike')
    for tStrike in StrikeTimes:
        plt.plot((tStrike, tStrike), (-180, 180), 'r-')
    plt.legend(loc='best')
    plt.xlabel('t')
    plt.ylabel('angle')
    plt.grid()
    plt.show()
    print('Period of small oscillations: {:.4} seconds'.format(small_oscillation(l_b)))
    print('Exact period: {:.4}'.format(Texact))
    print('Oddstruckness: {:.4}'.format(2*StrikeTimes[2]-StrikeTimes[1] - StrikeTimes[3]))

In [21]:
mylayout = Layout(width='80%', height='20px')
theta_0_float = FloatSlider(description=r'\(\theta_0\)', \
                        min=0.01, max=179.99, step=0.01, value=175, \
                        continuous_update=False, layout=mylayout)
l_b_float = FloatSlider(description=r'\(l_b\)', \
                        min=0.01, max=5, step=0.01, value=0.6, \
                        continuous_update=False, layout=mylayout)
l_c_float = FloatSlider(description=r'\(l_c\)', \
                        min=0.01, max=5, step=0.01, value=1, \
                        continuous_update=False, layout=mylayout)
c_float = FloatSlider(description=r'\(c\)', \
                      min=0.01, max=5, step=0.01, value=0.2, \
                      continuous_update=False, layout=mylayout)
alpha_float = FloatSlider(description=r'\(\alpha\)', \
                          min=-45, max=45, step=0.1, value=0, \
                          continuous_update=False, layout=mylayout)
interactive(update, theta_0=theta_0_float, l_b=l_b_float, l_c=l_c_float, c=c_float, alpha=alpha_float)

interactive(children=(FloatSlider(value=175.0, continuous_update=False, description='\\(\\theta_0\\)', layout=Layout(height='20px', width='80%'), max=179.99, min=0.01, step=0.01), FloatSlider(value=0.6, continuous_update=False, description='\\(l_b\\)', layout=Layout(height='20px', width='80%'), max=5.0, min=0.01, step=0.01), FloatSlider(value=1.0, continuous_update=False, description='\\(l_c\\)', layout=Layout(height='20px', width='80%'), max=5.0, min=0.01, step=0.01), FloatSlider(value=0.2, continuous_update=False, description='\\(c\\)', layout=Layout(height='20px', width='80%'), max=5.0, min=0.01, step=0.01), FloatSlider(value=0.0, continuous_update=False, description='\\(\\alpha\\)', layout=Layout(height='20px', width='80%'), max=45.0, min=-45.0), Output()), _dom_classes=('widget-interact',))